In [134]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from nltk import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import re

Tested for stop words, removing stopwords made the model worse.

using bigrams increases efficiency by a little, not much, but takes much longer time. might be better not to use it if efficiency is not great

- Increases efficiecny from 66.8 to 68.7

Now checking whether token_regex thing is better or using strip accesnts ascii is better, or using both is better.

After looking at feature names, I concluded that maybe using numbers might not be a good decision.
Will have to test the model by removing numbers or keeping them. what is the best way of removing numbers?


- using ascii gave us an increase from 68.7 to 69.15 with bigrams
- without bigrams it actually dropped from 66.8 to 66.45
- Using both ascii and regex without bigrams made it even worse to 66.4
- Using both dropped accuracy to 68.45
- Now testing none with bigrams: This gives me a 69.6 accuracy
- Then test a different token that completely gets rid of tokens with numbers in them
   - using  r'\b[^\d\W]+\b' which only selects alphanumeric words with underscores, accuracy was 68.7
   - using r'\b[^\d\W_]+\b' gives me 68.65
   - using min length 3 gave me 67.55
   - using min length 2 gave me 68.55
   - using r"\b[^\d\W]+\b|\b[0-9]+\b" as the token patter gave me an accuracy of 70.1 which is the best by far
   - Using the CountVectorizer gave much better accuracy than TfidfVectorizer - 61.5

testing on hyper-params: max_df and min_df
- max_df best is 1.0 itself (that is keeping in all words)
- min_df is also best at default. but at 3 gives 78.7. at 2 gives 78.35

Considerations of stemming and lemmatization
- stemming
- lemmatization

This is a 3% difference which is significant, hence maybe certain decisions imrpove the efficiency of bigrams.

checking sklearn and nltk: answer - create our own preprocessor for the CV.

Other decisions to consider:
Lower case everything or not? might miss importance of names if we convert to lower case
Consider removing all non-english characters

# Running LogisticRegressionCV

- With default setting (Cs=10 chosen between 1e-4 and 1e4)

Cs: array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])

Mean scores:
0    0.67795
1    0.73440
2    0.76475
3    0.76935
4    0.76600
5    0.76510
6    0.76380
7    0.76205
8    0.76130
9    0.76075

Max scores:
0    0.68450
1    0.74275
2    0.77175
3    0.78100
4    0.77900
5    0.78175
6    0.77950
7    0.77725
8    0.77675
9    0.77625

scores:

{1: array([[0.6785 , 0.72975, 0.76425, 0.7695 , 0.763  , 0.76   , 0.76   ,
         0.7595 , 0.759  , 0.7575 ],
        [0.68275, 0.73125, 0.76075, 0.7695 , 0.76575, 0.76725, 0.76425,
         0.76275, 0.762  , 0.762  ],
        [0.669  , 0.73175, 0.757  , 0.75625, 0.7545 , 0.7515 , 0.75125,
         0.74775, 0.7475 , 0.7475 ],
        [0.6845 , 0.74275, 0.77175, 0.781  , 0.779  , 0.78175, 0.7795 ,
         0.77725, 0.77675, 0.77625],
        [0.675  , 0.7365 , 0.77   , 0.7705 , 0.76775, 0.765  , 0.764  ,
         0.763  , 0.76125, 0.7605 ]])}

We know from previous tests that the accuracy when Cs = 1.0 is 78.7.

Hence peaking around there, now test on Cs values around 1.0.

78.4 accuracy on unseen data (when tested on validation set not used in CV)

- Cs=[0.01, 0.5, 0.75, 1.0, 1.25, 1.5, 2.0]

scores


{1: array([[0.76775, 0.76225, 0.7625 , 0.762  , 0.76175, 0.761  , 0.76025],
        [0.76825, 0.76625, 0.76575, 0.76725, 0.76725, 0.7675 , 0.76675],
        [0.75675, 0.7545 , 0.75325, 0.753  , 0.75275, 0.752  , 0.75175],
        [0.7755 , 0.77775, 0.7775 , 0.778  , 0.78   , 0.78   , 0.78075],
        [0.773  , 0.76775, 0.76775, 0.76725, 0.7665 , 0.76525, 0.76525]])}


Mean:

0    0.76825
1    0.76570
2    0.76535
3    0.76550
4    0.76565
5    0.76515
6    0.76495

Max:
0    0.77550
1    0.77775
2    0.77750
3    0.77800
4    0.78000
5    0.78000
6    0.78075

77.95 accuracy on unseen data.

- Now seeing as the highest mean was at 0.01 [1.00e-02, 4.00e-02, 7.00e-02, 9.00e-02]

scores:

{1: array([[0.76775, 0.769  , 0.76875, 0.76875],
        [0.76825, 0.77   , 0.76775, 0.76725],
        [0.75675, 0.758  , 0.75375, 0.75475],
        [0.7755 , 0.7805 , 0.78025, 0.7815 ],
        [0.773  , 0.76975, 0.76975, 0.76875]])}


Mean:
0    0.76825
1    0.76945
2    0.76805
3    0.76820

Max:
0    0.77550
1    0.78050
2    0.78025
3    0.78150

Accuracy on unseen data was 78.6%

-Now finally tuning on a good peak value of 4.00e-02 [3.50e-02, 3.75e-02, 4.00e-02, 4.25e-02, 4.50e-02]

In [136]:
vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r"\b[^\d\W]+\b|\b[0-9]+\b", min_df=3)

In [137]:
#pattern = re.compile(r'\b[^\d\W]+\b|\b[0-9]+\b')

In [138]:
fake_news_training = pd.read_csv('../data/fake_news/fake_news_train.csv')
fake_news_validation = pd.read_csv('../data/fake_news/fake_news_val.csv')
fake_news_training['text'][:10]

0    Indian fruit is so important to so many people...
1    FORT WORTH, Texas — Urú Inc. will hold a confe...
2    With three of the four new carriers, the Niger...
3    Let's start with the classic annual dividend r...
4    Following are some of the major events to have...
5    Posted\nAn empty classroom in rural communitie...
6    The whales we know today look nothing like the...
7    Local news anchor Richard Ray announced Thursd...
8    PROVO — In the 17 years that he’s been married...
9    Before a crowd of 4,000 people, Sen. Cory Book...
Name: text, dtype: object

In [139]:
#lemmatizer = WordNetLemmatizer()
#fake_news_training['text'] = fake_news_training['text'].apply(lambda corpus: ' '.join(' '.join([lemmatizer.lemmatize(word.lower()) for word in word_tokenize(sent) if word.isalpha()]) for sent in sent_tokenize(corpus)))
#fake_news_training['text'][:10]

In [140]:
#fake_news_validation['text'] = fake_news_validation['text'].apply(lambda corpus: ' '.join(' '.join([lemmatizer.lemmatize(word.lower()) for word in word_tokenize(sent) if word.isalpha()]) for sent in sent_tokenize(corpus)))


In [141]:
vectorizer.fit(fake_news_training['text'])


CountVectorizer(min_df=3, ngram_range=(1, 2),
                token_pattern='\\b[^\\d\\W]+\\b|\\b[0-9]+\\b')

In [142]:
vectorizer.get_feature_names_out()

array(['0', '0 0', '0 00', ..., 'únicos', 'œwe', 'œwe are'], dtype=object)

In [143]:
len(vectorizer.get_feature_names_out())

472940

In [144]:
training_features = vectorizer.transform(fake_news_training['text'])
validation_features = vectorizer.transform(fake_news_validation['text'])

LogisticRegressionCV(Cs=[0.01, 0.04, 0.07, 0.09], max_iter=1000)

In [151]:
clf = LogisticRegressionCV(max_iter=1000, Cs=[1.00e-02, 4.00e-02, 7.00e-02, 9.00e-02])
clf.fit(training_features, fake_news_training['label'])


LogisticRegressionCV(Cs=[0.01, 0.04, 0.07, 0.09], max_iter=1000)

In [152]:
y_pred = clf.predict(validation_features)
y_valid = fake_news_validation['label']

metrics.accuracy_score(y_valid, y_pred)

0.786

In [153]:
clf.scores_

{1: array([[0.76775, 0.769  , 0.76875, 0.76875],
        [0.76825, 0.77   , 0.76775, 0.76725],
        [0.75675, 0.758  , 0.75375, 0.75475],
        [0.7755 , 0.7805 , 0.78025, 0.7815 ],
        [0.773  , 0.76975, 0.76975, 0.76875]])}

In [154]:
clf.Cs_

array([0.01, 0.04, 0.07, 0.09])

In [155]:
CV = pd.DataFrame(data=clf.scores_[1])
CV.max(axis=0)



0    0.77550
1    0.78050
2    0.78025
3    0.78150
dtype: float64